In [445]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import math 
import time
from random import shuffle
import threading

In [446]:
from vose_sampler import VoseAlias
dist = {"H":0.2, "T":0.8}
VA = VoseAlias(dist)
VA.sample_n(size=10)

['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']

In [447]:
# filepath = "../edgelists/BlogCatalog-edgelist.txt"
filepath = "../edgelists/karate-small.txt"

embeddingsiterative = filepath+"-pairs.txt"

In [448]:
def parseEdgeList(graph_file, delimiter=" ", weighted=False, direction="undirected"):
    if(weighted == False):
        G = nx.read_edgelist(graph_file, delimiter=delimiter, nodetype=int)
    else:
        G = nx.read_edgelist(graph_file, delimiter=delimiter, nodetype=int, data=(('weight',float),))
    print(G.number_of_nodes(), G.number_of_edges(), " loaded from ", graph_file)
    if(direction == "undirected"):
        return G.to_undirected()
    else:
        return G

In [449]:
# G = parseEdgeList(filepath)

In [450]:
femb_recursive = open(embeddingsrecursive, 'w')


## Toy Graph

In [451]:
def createToyGraph():
    G = nx.Graph()
    #Small example
    G.add_nodes_from(["A","D","E","J","K","L","M",
                      "V","W","X","Y","Z","A1","B1","OX","DX"])
    G.add_edge("A", "D");G.add_edge("A", "E");
    G.add_edge("D", "J");G.add_edge("D", "K");
    G.add_edge("E", "L");G.add_edge("E", "M");
    G.add_edge("J", "V");G.add_edge("J", "W");
    G.add_edge("K", "X");
    G.add_edge("L", "Y");G.add_edge("L", "Z");
    G.add_edge("M", "A1");G.add_edge("M", "B1");
    G.add_edge("V", "OX");G.add_edge("V", "DX");

    # Draw graph
    #nx.draw(G, with_labels = True)
    #plt.show()
    return G

## Random Walk with window

In [452]:
def get_sample(arr, n_iter=None, sample_size=10, fast=True):
    if fast:
        # find the index we last sampled from
        start_idx = (n_iter * sample_size) % n
        if start_idx + sample_size >= n:
            # shuffle array if we have reached the end and repeat again
            np.random.shuffle(arr)
        return arr[start_idx:start_idx+sample_size] 
    else:
        return np.random.choice(arr, sample_size, replace=False)

In [453]:
def collect_samples(arr, sample_size, n_samples, fast=False):
    samples = np.zeros((n_samples + 1, sample_size), np.int32)
    #print("samples ->", samples)
    
    for sample_n in range(0, n_samples):
        sample = get_sample(arr, n_iter=sample_n, sample_size=sample_size, fast=fast)
        samples[sample_n] = sample   
    return samples


In [454]:
n_samples = 1
sample_size = 5
n_iter = 1
arr = np.array([1,2,3,4,5,6,7,8,9,0]).astype(np.int64)

In [455]:
get_sample(arr, n_iter, sample_size, fast=False)

array([7, 6, 5, 0, 9])

In [456]:
collect_samples(arr, sample_size, 1, fast=False)[0]

array([9, 0, 2, 1, 6], dtype=int32)

In [457]:
def getNodeContextSets(graph):
    node_context = {}
    for node in graph:
        node_context[node] = (set(), 0)
    return node_context
node_context = getNodeContextSets(G)

In [458]:
def chooseNodes(list_nodes, sample_size):
    #n_iter = 1
    #return np.random.choice(list_nodes, n, replace=False)
    #return random.sample(population=list(list_nodes), k=sample_size)
    return collect_samples(list_nodes, sample_size, 1, fast=False)[0]

In [459]:
def getPerNodeBudget(numNodes, budget):
    return math.floor(budget/numNodes)

In [460]:
def storeContextPairs(context_pair, budget, context_pairs):
    if context_pair not in context_pairs:
        context_pairs[context_pair] = budget
    else:
        context_pairs[context_pair] = context_pairs[context_pair] +  budget

In [461]:
def updateSetCount(node, context_node, context_budget):
    context_set = node_context[node][0]
    context_pair_count = node_context[node][1]
    context_set.add(context_node)
    context_pair_count = context_pair_count + context_budget
    node_context[node] = (context_set, context_pair_count)

In [462]:
def updateContextPairs(window, context_pairs, windowSize):    
#     lastNode = window[-1]
#     labelOfLastNode, budgetOfLastNode = lastNode
    window_len = len(window)
    middlenode_index = math.floor(window_len / 2)
    middlenode, budgetofmiddlenode = window[middlenode_index]
    for node in window:
        context_node = node[0]
        if context_node == middlenode:
            continue
#         context_pair1 = (context_node,middlenode)
        context_pair = (middlenode, context_node)
        
        
        updateSetCount(middlenode, context_node, budgetofmiddlenode)
        
#         set2 = node_context[middlenode][0]
#         count2 = node_context[middlenode][1]
#         updateSetCount(set2, context_node, count2, middlenode, budgetofmiddlenode)
    
        storeContextPairs(context_pair, budgetofmiddlenode, context_pairs)
#         storeContextPairs(context_pair2, budgetofmiddlenode, context_pairs)
    

In [463]:
def addNewNodeToWindow(window, vertex, budget):
    window.append([vertex,budget])
    return window

### Test case 

In [464]:
# Parse the toy graph
G = createToyGraph()

# Set the toy parameters 
WALK_LENGHT = 4
BUDGET = 4
WINDOW_SIZE = 1
WINDOW_SIZE = WINDOW_SIZE*2+1
WINDOW=[]

### Actual case

In [465]:
# Parse the actual data
G = parseEdgeList(filepath)

11 20  loaded from  ../edgelists/karate-small.txt


In [466]:
# Set the actual parameters 
WALK_LENGHT = 3
BUDGET = 1
WINDOW_SIZE = 1

WINDOW_SIZE = WINDOW_SIZE*2+1
WINDOW=[]
random.seed(1)

In [467]:
def getAdjList(graph):
    adjdict = {}
    for neighbor in graph:
        neighbors = [n for n in G.neighbors(neighbor)]
#         print(neighbors)
        num_neihbors = len(neighbors)
        adjdict[neighbor] = random.sample(neighbors, num_neihbors)
#         degreedict[neighbor] = num_neihbors
    return adjdict
adjdict = getAdjList(G)

In [468]:
def sampleFromExistingContext(node, budget):
        context_nodes, num_walks = node_context[node]
        num_nodes = len(context_nodes)
        if num_nodes == 0:
            return False
        m = getPerNodeBudget(num_nodes, budget)
        remainder = budget - (m * num_nodes)
        for context_node in context_nodes:
            context_pair1 = (context_node, node)
            context_pair2 = (node, context_node)
            count = 0
            if context_pair1 in context_pairs:
                count1 = context_pairs[context_pair1]
            if count1 != 0 and num_walks > 0:
                budget_for_this_node = math.floor((float(count1) / float(num_walks)) * budget)
            else:
                budget_for_this_node = m
                if(remainder > 0):
                    budget_for_this_node = budget_for_this_node + 1
                    remainder = remainder - 1
            budget = budget - budget_for_this_node
            if budget_for_this_node > 0:
                storeContextPairs(context_pair1, budget_for_this_node, context_pairs)
                storeContextPairs(context_pair2, budget_for_this_node, context_pairs)
            else:
                break
        return True

In [469]:
def sampleFromExistingProbabilityDist(context_nodes, num_walks_passing, node, budget):
    num_nodes = len(context_nodes)
    if num_nodes == 0 or num_walks_passing == 0:
        return False
    m = getPerNodeBudget(num_nodes, budget)
    remainder = budget - (m * num_nodes)
    dist = {}
    for context_node in context_nodes:
        context_pair = (node, context_node)
        context_pair_count = context_pairs[context_pair]
        context_pair_prob = float(context_pair_count) / float(num_walks_passing)
        dist[context_pair] = context_pair_prob
    VA = VoseAlias(dist)
    sampled_budgets = VA.sample_n(size=budget)
    for sample_pair in sampled_budgets:
        storeContextPairs(sample_pair, 1, context_pairs)
        updateSetCount(sample_pair[0], sample_pair[1], 1)
    return True

In [470]:
def BFSRandomWalkWindow(graph, start, queue, context_pairs, window_size, walk_lenght, EPSILON):
    while queue:
        vertex, budget, current_walk_lenght, window = queue.pop(0)
        randval = random.random()
        #TODO change the probability inversely proportional to number of walks which passed through it
        context_nodes, num_walks_passing = node_context[vertex]
        if randval > EPSILON:
#             print("sampling from existing")
            if sampleFromExistingProbabilityDist(context_nodes, num_walks_passing, vertex, budget) == True:
                continue
#         print("continuing")
        vertex_neighbors = adjdict[vertex]
        num_neighbors = len(vertex_neighbors)
        m = getPerNodeBudget(num_neighbors, budget)
        remainder = budget - (m * num_neighbors)
        current_walk_lenght += 1
        for neighbor in vertex_neighbors:
            budget_for_this_node = m 
            if(remainder > 0):
                budget_for_this_node = budget_for_this_node + 1
                remainder = remainder - 1
#                 num_chosen_nodes = num_chosen_nodes + 1
            if(budget_for_this_node > 0):
                if len(window) == window_size:
                    window.pop(0)
                addNewNodeToWindow(window, neighbor, budget_for_this_node) 
                updateContextPairs(window, context_pairs, window_size)    
                if current_walk_lenght < walk_lenght:
                    queue.append((neighbor, budget_for_this_node, current_walk_lenght, list(window)))
                window.pop(-1)


In [471]:
context_pairs = {}
counter = 0
print("Running BFSRandomWalkWindow...")
nodedegree = sorted(G.degree, key=lambda x: x[1], reverse=True)
nodes = [n[0] for n in nodedegree]
start = time.time()
for startvertex in nodes:
    WINDOW = [[startvertex, BUDGET]]
    counter +=1
    queue = [(startvertex, BUDGET, 0, WINDOW)]
    BFSRandomWalkWindow(G, startvertex, queue, context_pairs, WINDOW_SIZE, WALK_LENGHT, 1)
    if counter % 1000 == 0:
        end = time.time()
        t = end - start
        start = time.time()
        print("performance for "+str(counter)+" random walks : ",t)
#         break
# print("performance for 100 random walks : ",t)

Running BFSRandomWalkWindow...


### Writing to file

In [472]:
print("Writing context pairs to file...")    
#Writing to file 

femb_iterative = open(embeddingsiterative, 'w')
for (key, value) in context_pairs.items():
    femb_iterative.write(str(key[0]) + " " + str(key[1]) + " " + str(value) + "\n" )
femb_iterative.close()
end = time.time()
t = end - start
print("performance for 100 random walks : ",t)

Writing context pairs to file...
performance for 100 random walks :  0.018815994262695312


In [473]:
x = np.array([3, 4, 2, 7, 5, 6, 1])
x[np.argpartition(x, (0,1,))]
#Changes the given index(0) with the minimum element in the array 

array([1, 2, 4, 7, 5, 6, 3])